# 申购单审批流程

## 1）issue

[申购单-简道云触发器](https://dev.eastwestec.com/projects/cecp/wiki/%E7%94%B3%E8%B4%AD%E5%8D%95-%E7%AE%80%E9%81%93%E4%BA%91%E8%A7%A6%E5%8F%91%E5%99%A8)

## 2）分支

weld_backend: `feature/0513_purchase_demand`  
switch: `feature/0513_purchase_demand`  

## 3）服务

### 3.1）weld_backend 与 switch 服务启动

### 3.2）celery

weld_backend: `celery -A celery_app.celery worker`  
switch: `celery -A celery_app.celery worker`  

## 4）代码解析

### 4.1）switch（关于 notification 类型的任务的定义流程）

* a. 创建task：`mapi.modules.notification.jiandaoyun.purchase_demand.tasks.purchase_demand`

```python
@Handle.register(event_type='purchase_demand_flow')
@celery.task(bind=True)
def purchase_demand(self, payload, platform):
    """简道云 -> cecp 创建采购任务"""
    print payload, type(payload)
```

---

* b. 注册 task（celery task）：`mapi.celery_imports.CELERY_IMPORTS`

* c. 定义 task 的事件触发类型：`mapi.services.jiandaoyun_sdk.parser.JianDaoYunPurchaseDemandEntryParser`

* d. 注册事件类型：`mapi.adapters.handle.JianDaoYunNotificationHandle#event_types`

###  4.2）weld_backend

`bapi.modules.purchase.case.views.PurchaseDemandAPIView#external_approval`


## 5）测试

cecp / switch 可开启 debug 模式。

### 5.1）weld_backend 查询采购任务是否创建

`python manager.py shell`  # weld_backend

`PurchaseDemand.query.join(SKU).filter(PurchaseDemand.sku_id == SKU.id, SKU.sku == "LP250021BU").all()`


### 5.2）模拟简道云发起同步请求，switch 将从简道云获取的数据发送到cecp接口
  
`python manager.py shell`  # switch

```python
def data():
	import json
	payload = {
		'purchase_demands': [
			{
				'createTime': u'2019-05-07T06:50:26.519Z',
   				'demand_person': {'demand_person': u'卢艳'},
   				'destination': {'destination': u'YD-EU-FBA仓'},
   				'eta': u'2019-05-13T16:00:00.000Z',
   				'final_warehouse': {'final_warehouse': u'QS-EU-FBA仓'},
   				'quantity': 123,
   				# 'remark': u'\u9f13\u6363\u9f13\u6363',
   				'team_business_division': {'team_business_division': u'大家居'},
   				'sku': {'sku': u'LP250021BU'}
   			}
   		],
   		'remarks': [u'\u9f13\u6363\u9f13\u6363']
   	}
	data = json.dumps(payload)
	return data

from mapi.services.app_clients import cecp_client
response = cecp_client.requests('POST', 'purchase/v1/demands/jdy_create_batch', data=data())
```
